## 2. Tensorflow Low Level APIs

### 2.5. 保存和恢复

**保存和恢复变量**

---

（稍后更新）

**保存和恢复模型**

---

使用`SavedModel`保存和加载模型--包括模型的变量、图（数据流图）和图的元数据（关于图的描述信息）。SavedModel是一种语言独立的、可恢复的、密封的序列化格式，使higher-level systems和tools可以创建、使用和转换TensorFlow模型。TensorFlow提供了多种与SavedModel交互的方式，包括 [tf.saved_model API](https://www.tensorflow.org/api_docs/python/tf/saved_model?hl=zh-cn)、[tf.estimator.Estimator](https://www.tensorflow.org/api_docs/python/tf/estimator/Estimator?hl=zh-cn)和命令行界面。

**构建和加载SavedModel**

---

**简单保存**

创建`SavedModel`(使用`SavedModel`保存模型)的最简单方法是使用[tf.saved_model.simple_save]()函数：

```
simple_save(session,
            export_dir,
            inputs={"x": x, "y": y},
            outputs={"z": z})
```

这样可以配置`SavedModel`，使其能够通过[TensorFlow Serving](https://www.tensorflow.org/serving/serving_basic?hl=zh-cn)进行加载，并支持[Predict API](https://github.com/tensorflow/serving/blob/master/tensorflow_serving/apis/predict.proto)。要访问 classify API、regress API 或 multi-inference API，（更多RESTFul API，详见[REST API](https://www.tensorflow.org/serving/api_rest?hl=zh-cn)）请使用手动`SavedModel` builder API或[tf.estimator.Estimator](https://www.tensorflow.org/api_docs/python/tf/estimator/Estimator?hl=zh-cn)。


注释：
1. tensorflow.saved_model.simple_save:
```
tensorflow.saved_model.simple_save = simple_save(session, export_dir, inputs, outputs, legacy_init_op=None)
    Convenience function to build a SavedModel suitable for serving.

    In many common cases, saving models for serving will be as simple as:

        simple_save(session,
                    export_dir,
                    inputs={"x": x, "y": y},
                    outputs={"z": z})

    Although in many cases it's not necessary to understand all of the many ways
        to configure a SavedModel, this method has a few practical implications:
      - It will be treated as a graph for inference / serving (i.e. uses the tag
        `tag_constants.SERVING`)
      - The SavedModel will load in TensorFlow Serving and supports the
        [Predict
        API](https://github.com/tensorflow/serving/blob/master/tensorflow_serving/apis/predict.proto).
        To use the Classify, Regress, or MultiInference APIs, please
        use either
        [tf.Estimator](https://www.tensorflow.org/api_docs/python/tf/estimator/Estimator)
        or the lower level
        [SavedModel
        APIs](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/python/saved_model/README.md).
      - Some TensorFlow ops depend on information on disk or other information
        called "assets". These are generally handled automatically by adding the
        assets to the `GraphKeys.ASSET_FILEPATHS` collection. Only assets in that
        collection are exported; if you need more custom behavior, you'll need to
        use the
        [SavedModelBuilder](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/python/saved_model/builder.py).

    More information about SavedModel and signatures can be found here:
    https://github.com/tensorflow/tensorflow/blob/master/tensorflow/python/saved_model/README.md.

    Args:
      session: The TensorFlow session from which to save the meta graph and
          variables.
      export_dir: The path to which the SavedModel will be stored.
      inputs: dict mapping string input names to tensors. These are added
          to the SignatureDef as the inputs.
      outputs:  dict mapping string output names to tensors. These are added
          to the SignatureDef as the outputs.
      legacy_init_op: Legacy support for op or group of ops to execute after the
          restore op upon a load.
```

**手动构建SavedModel**

如果[tf.saved_model.simple_save](https://www.tensorflow.org/api_docs/python/tf/saved_model/simple_save?hl=zh-cn)不能满足你的使用需求（需要classify api或者配置`SavedModel`的TAG等等），请手动使用[tf.saved_model.builder](https://www.tensorflow.org/api_docs/python/tf/saved_model/builder)创建`SavedModel`。

[tf.saved_model.builder.SavedModelBuilder](https://www.tensorflow.org/api_docs/python/tf/saved_model/builder/SavedModelBuilder?hl=zh-cn)类提供了保存多个`MetaGraphDef`的功能。**MetaGraph**是一种**数据流图**，并包含相关**变量**、资源assets和签名signature（**图的元数据**）。
+ `MetaGraphDef`是`MetaGraph`的协议缓冲区表示法。
+ `signature_def_map`--**签名signature**：一组与图有关的输入和输出。
+ `assets_collection`--**资源assets**：在首次添加`MetaGraphDef`是提供资源assets，则可把这些assets保存并写入或复制到磁盘；如果多个`MetaGraphDef`与同名资源相关联，则只保留首个版本。If multiple MetaGraphDefs are associated with an asset of the same name, only the first version is retained.
+ `sess`--**数据流图进程**：包含着需要保存的数据流图的进程；
+ `tag`--**标签**：使用用户指定的标签对每个添加到`SavedModel`的`MetaGraphDef`进行标注。这些标签提供了一种方法来识别要加载和恢复的特定 `MetaGraphDef`，以及共享的变量和资源子集。这些标签一般会标注`MetaGraphDef`的功能（例如服务或训练），有时也会标注特定的硬件方面的信息（如 GPU）。

例如，以下代码展示了使用 SavedModelBuilder 构建 SavedModel 的典型方法：

```
export_dir = ...
...
builder = tf.saved_model.builder.SavedModelBuilder(export_dir)
with tf.Session(graph=tf.Graph()) as sess:
  ...
  builder.add_meta_graph_and_variables(sess,
                                       [tag_constants.TRAINING],
                                       signature_def_map=foo_signatures,
                                       assets_collection=foo_assets,
                                       strip_default_attrs=True)
...
# Add a second MetaGraphDef for inference.
with tf.Session(graph=tf.Graph()) as sess:
  ...
  builder.add_meta_graph([tag_constants.SERVING], strip_default_attrs=True)
...
builder.save()
```

注释：
1. tensorflow.saved_model.builder.SavedModelBuilder.add_meta_graph_and_variables():
```
tensorflow.saved_model.builder.SavedModelBuilder.add_meta_graph_and_variables = add_meta_graph_and_variables(self, sess, tags, signature_def_map=None, assets_collection=None, legacy_init_op=None, clear_
devices=False, main_op=None, strip_default_attrs=False, saver=None)
    Adds the current meta graph to the SavedModel and saves variables. (deprecated arguments)

    SOME ARGUMENTS ARE DEPRECATED. They will be removed in a future version.
    Instructions for updating:
    Pass your op to the equivalent parameter main_op instead.

    Creates a Saver to save the variables from the provided session. Exports the
    corresponding meta graph def. This function assumes that the variables to be
    saved have been initialized. For a given `SavedModelBuilder`, this API must
    be called exactly once and for the first meta graph to save. For subsequent
    meta graph defs to be added, the `add_meta_graph()` API must be used.

    Args:
      sess: The TensorFlow session from which to save the meta graph and
        variables.
      tags: The set of tags with which to save the meta graph.
      signature_def_map: The map of signature def map to add to the meta graph
        def.
      assets_collection: Assets collection to be saved with SavedModel.
      legacy_init_op: Legacy support for op or group of ops to execute after the
          restore op upon a load. Deprecated; please use main_op instead.
      clear_devices: Set to true if the device info on the default graph should
          be cleared.
      main_op: Op or group of ops to execute when the graph is loaded. Note
          that when the main_op is specified it is run after the restore op at
          load-time.
      strip_default_attrs: Boolean. If `True`, default-valued attributes will be
        removed from the NodeDefs. For a detailed guide, see
        [Stripping Default-Valued Attributes](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/python/saved_model/README.md#stripping-default-valued-attributes).
      saver: An instance of tf.train.Saver that will be used to export the
        metagraph and save variables. If None, a sharded Saver that restores
        all variables will be used.
```
2. tensorflow.saved_model.builder.SavedModelBuilder()
```
tensorflow.saved_model.builder.SavedModelBuilder = class SavedModelBuilder(builtins.object)
 |  Builds the `SavedModel` protocol buffer and saves variables and assets.
 |
 |  The `SavedModelBuilder` class provides functionality to build a `SavedModel`
 |  protocol buffer. Specifically, this allows multiple meta graphs to be saved as
 |  part of a single language-neutral `SavedModel`, while sharing variables and
 |  assets.
 |
 |  To build a SavedModel, the first meta graph must be saved with variables.
 |  Subsequent meta graphs will simply be saved with their graph definitions. If
 |  assets need to be saved and written or copied to disk, they can be provided
 |  when the meta graph def is added. If multiple meta graph defs are associated
 |  an asset of the same name, only the first version is retained.
 |
 |  Each meta graph added to the SavedModel must be annotated with tags. The tags
 |  provide a means to identify the specific meta graph to load and restore, along
 |  with the shared set of variables and assets.
 |
 |  Typical usage for the `SavedModelBuilder`:
 |  ``python
 |  ...
 |  builder = tf.saved_model.Builder(export_dir)
 |
 |  with tf.Session(graph=tf.Graph()) as sess:
 |    ...
 |    builder.add_meta_graph_and_variables(sess,
 |                                    ["foo-tag"],
 |                                    signature_def_map=foo_signatures,
 |                                    assets_collection=foo_assets)
 |  ...
 |
 |  with tf.Session(graph=tf.Graph()) as sess:
 |    ...
 |    builder.add_meta_graph(["bar-tag", "baz-tag"])
 |  ...
 |
 |  builder.save()
 |  ``
 |
 |  Methods defined here:
 |
 |  __init__(self, export_dir)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |
 |  add_meta_graph(self, tags, signature_def_map=None, assets_collection=None, legacy_init_op=None, clear_devices=False, main_op=None, strip_default_attrs=False, saver=None)
 |      Adds the current meta graph to the SavedModel. (deprecated arguments)
 |
 |      SOME ARGUMENTS ARE DEPRECATED. They will be removed in a future version.
 |      Instructions for updating:
 |      Pass your op to the equivalent parameter main_op instead.
 |
 |      Creates a Saver in the current scope and uses the Saver to export the meta
 |      graph def. Invoking this API requires the `add_meta_graph_and_variables()`
 |      API to have been invoked before.
 |
 |      Args:
 |        tags: The set of tags to annotate the meta graph def with.
 |        signature_def_map: The map of signature defs to be added to the meta graph
 |            def.
 |        assets_collection: Assets collection to be saved with SavedModel. Note
 |            that this collection should be a subset of the assets saved as part of
 |            the first meta graph in the SavedModel.
 |        legacy_init_op: Legacy support for op or group of ops to execute after the
 |            restore op upon a load. Deprecated; please use main_op instead.
 |        clear_devices: Set to true if the device info on the default graph should
 |            be cleared.
 |        main_op: Op or group of ops to execute when the graph is loaded. Note
 |            that when the main_op is specified it is run after the restore op at
 |            load-time.
 |        strip_default_attrs: Boolean. If `True`, default-valued attributes will be
 |          removed from the NodeDefs. For a detailed guide, see
 |          [Stripping Default-Valued Attributes](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/python/saved_model/README.md#stripping-default-valued-attributes).
 |        saver: An instance of tf.train.Saver that will be used to export the
 |          metagraph. If None, a sharded Saver that restores all variables will
 |          be used.
 |
 |      Raises:
 |        AssertionError: If the variables for the SavedModel have not been saved
 |            yet, or if the graph already contains one or more legacy init ops.
 |
 |  add_meta_graph_and_variables(self, sess, tags, signature_def_map=None, assets_collection=None, legacy_init_op=None, clear_devices=False, main_op=None, strip_default_attrs=False, saver=None)
 |      Adds the current meta graph to the SavedModel and saves variables. (deprecated arguments)
 |
 |      SOME ARGUMENTS ARE DEPRECATED. They will be removed in a future version.
 |      Instructions for updating:
 |      Pass your op to the equivalent parameter main_op instead.
 |
 |      Creates a Saver to save the variables from the provided session. Exports the
 |      corresponding meta graph def. This function assumes that the variables to be
 |      saved have been initialized. For a given SavedModelBuilder, this API must
 |      be called exactly once and for the first meta graph to save. For subsequent
 |      meta graph defs to be added, the add_meta_graph() API must be used.
 |
 |      Args:
 |        sess: The TensorFlow session from which to save the meta graph and
 |          variables.
 |        tags: The set of tags with which to save the meta graph.
 |        signature_def_map: The map of signature def map to add to the meta graph
 |          def.
 |        assets_collection: Assets collection to be saved with SavedModel.
 |        legacy_init_op: Legacy support for op or group of ops to execute after the
 |            restore op upon a load. Deprecated; please use main_op instead.
 |        clear_devices: Set to true if the device info on the default graph should
 |            be cleared.
 |        main_op: Op or group of ops to execute when the graph is loaded. Note
 |            that when the main_op is specified it is run after the restore op at
 |            load-time.
 |        strip_default_attrs: Boolean. If True, default-valued attributes will be
 |          removed from the NodeDefs. For a detailed guide, see
 |          [Stripping Default-Valued Attributes](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/python/saved_model/README.md#stripping-default-valued-attributes).
 |        saver: An instance of tf.train.Saver that will be used to export the
 |          metagraph and save variables. If None, a sharded Saver that restores
 |          all variables will be used.
 |
 |  save(self, as_text=False)
 |      Writes a SavedModel protocol buffer to disk.
 |
 |      The function writes the SavedModel protocol buffer to the export directory
 |      in serialized format.
 |
 |      Args:
 |        as_text: Writes the SavedModel protocol buffer in text format to disk.
 |
 |      Returns:
 |        The path to which the SavedModel protocol buffer was written.
 |
 |  ----------------------------------------------------------------------
```
3. [tag_constants.py](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/python/saved_model/tag_constants.py)中的tag：`SERVING`, `TRAINING`, `EVAL`, `GPU`, `TPU`;

**Forward compatibility via strip_default_attrs=True**

Following the guidance below gives you forward compatibility only if the set of Ops has not changed.

The [tf.saved_model.builder.SavedModelBuilder](https://www.tensorflow.org/api_docs/python/tf/saved_model/builder/SavedModelBuilder) class allows users to control whether default-valued attributes must be stripped from the [NodeDefs](https://www.tensorflow.org/extend/tool_developers/index#nodes) while adding a meta graph to the SavedModel bundle. Both [tf.saved_model.builder.SavedModelBuilder.add_meta_graph_and_variables](https://www.tensorflow.org/api_docs/python/tf/saved_model/builder/SavedModelBuilder#add_meta_graph_and_variables) and [tf.saved_model.builder.SavedModelBuilder.add_meta_graph](https://www.tensorflow.org/api_docs/python/tf/saved_model/builder/SavedModelBuilder#add_meta_graph) methods accept a Boolean flag `strip_default_attrs` that controls this behavior.

If `strip_default_attrs` is False, the exported [tf.MetaGraphDef](https://www.tensorflow.org/api_docs/python/tf/MetaGraphDef) will have the default valued attributes in all its [tf.NodeDef](https://www.tensorflow.org/api_docs/python/tf/NodeDef) instances. This can break forward compatibility with a sequence of events such as the following:

+ An existing Op (Foo) is updated to include a new attribute (T) with a default (bool) at version 101.
+ A model producer such as a "trainer binary" picks up this change (version 101) to the OpDef and re-exports an existing model that uses Op Foo.
+ A model consumer (such as [Tensorflow Serving](https://www.tensorflow.org/serving)) running an older binary (version 100) doesn't have attribute T for Op Foo, but tries to import this model. The model consumer doesn't recognize attribute T in a NodeDef that uses Op Foo and therefore fails to load the model.
+ By setting strip_default_attrs to True, the model producers can strip away any default valued attributes in the NodeDefs. This helps ensure that newly added attributes with defaults don't cause older model consumers to fail loading models regenerated with newer training binaries.

See [compatibility guidance](https://www.tensorflow.org/guide/version_compat) for more information.



**在Python中加载SavedModel**

Python版本的SavedModel [tf.saved_model.loader](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/cc/saved_model/loader.h)提供加载和恢复SavedModel的功能。加载操作需要以下信息：

+ 用于存储恢复流图的进程--The session in which to restore the graph definition and variables.
+ 用于标识要加载的MetaGraphDef的标签--The tags used to identify the MetaGraphDef to load.
+ SavedModel的位置--The location (directory) of the SavedModel.

加载后，作为特定 MetaGraphDef 的一部分提供的变量、资源和签名子集将恢复到提供的会话中。Upon a load, the subset of variables, assets, and signatures supplied as part of the specific MetaGraphDef will be restored into the supplied session.

```
export_dir = ...
...
with tf.Session(graph=tf.Graph()) as sess:
  tf.saved_model.loader.load(sess, [tag_constants.TRAINING], export_dir)
  ...
``` 

注释：
1. tensorflow.saved_model.loader.load()
```
tensorflow.saved_model.loader.load = load(sess, tags, export_dir, import_scope=None, **saver_kwargs)
    Loads the model from a SavedModel as specified by tags.

    Args:
      sess: The TensorFlow session to restore the variables.
      tags: Set of string tags to identify the required MetaGraphDef. These should
          correspond to the tags used when saving the variables using the
          SavedModel `save()` API.
      export_dir: Directory in which the SavedModel protocol buffer and variables
          to be loaded are located.
      import_scope: Optional `string` -- if specified, prepend this string
          followed by '/' to all loaded tensor names. This scope is applied to
          tensor instances loaded into the passed session, but it is *not* written
          through to the static `MetaGraphDef` protocol buffer that is returned.
      **saver_kwargs: Optional keyword arguments passed through to Saver.

    Returns:
      The `MetaGraphDef` protocol buffer loaded in the provided session. This
      can be used to further extract signature-defs, collection-defs, etc.

    Raises:
      RuntimeError: MetaGraphDef associated with the tags cannot be found.
```

**在C++中加载SavedModel**

此处指出只是为了说明SavedModel的语言独立性，详见<https://www.tensorflow.org/guide/saved_model?hl=en>;



**在 TensorFlow Serving 中加载和提供 SavedModel**

您可以使用 TensorFlow Serving Model Server 二进制文件轻松加载和提供 SavedModel。参阅[此处](https://www.tensorflow.org/serving/setup#installing_using_apt-get)的说明，了解如何安装服务器，或根据需要创建服务器。

一旦您的 Model Server 就绪，请运行以下内容：

```
tensorflow_model_server --port=port-numbers --model_name=your-model-name --model_base_path=your_model_base_path
```

将 port 和 model_name 标记设为您所选的值。model_base_path 标记应为基本目录，每个版本的模型都放置于以数字命名的子目录中。如果您的模型只有一个版本，只需如下所示地将其放在子目录中即可： * 将模型放入 /tmp/model/0001 * 将 model_base_path 设为 /tmp/model

将模型的不同版本存储在共用基本目录的子目录中（以数字命名）。例如，假设基本目录是`/tmp/model`。如果您的模型只有一个版本，请将其存储在 `/tmp/model/0001`中。如果您的模型有两个版本，请将第二个版本存储在`/tmp/model/0002`中，以此类推。将`--model-base_path`标记设为基本目录（在本例中为`/tmp/model`）。TensorFlow Model Server 将在该基本目录的最大编号的子目录中提供模型。

**标准 MetaGraphDef 标签 Standard MetaGraphDef Tags**

您可以使用一组标签集合唯一标识保存在SavedModel中的`MetaGraphDef`。tensorflow定义的常用标签的子集合如下：

+ [Python](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/python/saved_model/tag_constants.py) (十分重要)；
+ [C++](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/cc/saved_model/tag_constants.h)


**标准 SignatureDef 常量Standard SignatureDef constants**

[SignatureDef](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/core/protobuf/meta_graph.proto)是一个协议缓冲区，用于定义图所支持计算的签名。Tensorflow定义的常用的输入键、输出键和方法名称定义如下（因为signature_def_map是一个字典，而如下文件提供了常用的标准输入、输出键key和方法名称常量）：

+ [Python](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/python/saved_model/signature_constants.py) (十分重要)；
+ [C++](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/cc/saved_model/signature_constants.h)


结合上述文件，看官方的提供一个tensorflow模型的教程[Serving a TensorFlow Model](https://github.com/tensorflow/serving/blob/master/tensorflow_serving/g3doc/serving_basic.md)和[示例代码](https://github.com/tensorflow/serving/blob/master/tensorflow_serving/example/mnist_saved_model.py)，一目了然

上述链接的注释：
1. tensorflow.saved_model.build_signature_def:
```
tensorflow.saved_model.build_signature_def = build_signature_def(inputs=None, outputs=None, method_name=None)
    Utility function to build a SignatureDef protocol buffer.

    Args:
      inputs: Inputs of the SignatureDef defined as a proto map of string to
          tensor info.
      outputs: Outputs of the SignatureDef defined as a proto map of string to
          tensor info.
      method_name: Method name of the SignatureDef as a string.

    Returns:
      A SignatureDef protocol buffer constructed based on the supplied arguments.
```
2. tensorflow.saved_model.build_tensor_info:
```
tensorflow.saved_model.build_tensor_info = build_tensor_info(tensor)
    Utility function to build TensorInfo proto.

    Args:
      tensor: Tensor or SparseTensor whose name, dtype and shape are used to
          build the TensorInfo. For SparseTensors, the names of the three
          constitutent Tensors are used.

    Returns:
      A TensorInfo protocol buffer constructed based on the supplied argument.
```


**搭配Estimator使用SavedModel**

---

（未完待续）


**使用命令行CLI检查并执行SavedModel**

---

（未完待续）



---

教程链接：<https://www.tensorflow.org/guide/saved_model?hl=en>